In [2]:
!pip install -q langchain_huggingface langchain-core requests

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 1.0.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.5 which is incompatible.


In [6]:
!pip install langchain langchain-core langchain-community pydantic  langchain_experimental

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.5
    Uninstalling langchain-core-1.

In [3]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.5 MB/s eta 0:00:00


In [4]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

# 1) Creating Tool

In [5]:
@tool
def multiply(a: int, b: int)-> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a*b

In [9]:
result = multiply.invoke({"a":3, "b": 3})
print(result)

9


# 2) Tool Binding

In [10]:
llm = HuggingFaceEndpoint(
    repo_id = "Qwen/Qwen3-4B-Instruct-2507",
    task = "text-generation",
)
model = ChatHuggingFace(llm = llm)

In [13]:
llmWithTools = model.bind_tools([multiply])

 # 3) Tool Calling

In [14]:
llmWithTools.invoke("Hi how are you")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


AIMessage(content="I'm functioning well, thank you for asking! How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 167, 'total_tokens': 185}, 'model_name': 'Qwen/Qwen3-4B-Instruct-2507', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--eb13455b-df60-4c3a-b587-82bf3ea08c78-0', usage_metadata={'input_tokens': 167, 'output_tokens': 18, 'total_tokens': 185})

In [39]:
query = HumanMessage("can you multiply 2 with 30")

In [43]:
messages = [query]

In [44]:
result = llmWithTools.invoke(messages)
print(result)

content='' additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 2, "b": 30}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-ca922a3cc89d4e8aa001f281ba80d6cf', 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 172, 'total_tokens': 198}, 'model_name': 'Qwen/Qwen3-4B-Instruct-2507', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--d3c499d9-d2dd-4fd7-9aa0-6e6fc9127f91-0' tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 30}, 'id': 'chatcmpl-tool-ca922a3cc89d4e8aa001f281ba80d6cf', 'type': 'tool_call'}] usage_metadata={'input_tokens': 172, 'output_tokens': 26, 'total_tokens': 198}


In [45]:
messages.append(result)

In [46]:
messages

[HumanMessage(content='can you multiply 2 with 30', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 2, "b": 30}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-ca922a3cc89d4e8aa001f281ba80d6cf', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 172, 'total_tokens': 198}, 'model_name': 'Qwen/Qwen3-4B-Instruct-2507', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d3c499d9-d2dd-4fd7-9aa0-6e6fc9127f91-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 30}, 'id': 'chatcmpl-tool-ca922a3cc89d4e8aa001f281ba80d6cf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 172, 'output_tokens': 26, 'total_tokens': 198})]

In [47]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 2, 'b': 30},
 'id': 'chatcmpl-tool-ca922a3cc89d4e8aa001f281ba80d6cf',
 'type': 'tool_call'}

# 4) Tool Execution

In [48]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 2, 'b': 30},
 'id': 'chatcmpl-tool-ca922a3cc89d4e8aa001f281ba80d6cf',
 'type': 'tool_call'}

In [49]:
result.tool_calls[0]['args']

{'a': 2, 'b': 30}

In [50]:
multiply.invoke(result.tool_calls[0]['args'])

60

In [51]:
toolResult = multiply.invoke({'name': 'multiply',
 'args': {'a': 2, 'b': 3},
 'id': 'chatcmpl-tool-6710759eb1fb4425a4b3185f4ecf74bc',
 'type': 'tool_call'})

In [52]:
messages.append(toolResult)

In [53]:
messages

[HumanMessage(content='can you multiply 2 with 30', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 2, "b": 30}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-ca922a3cc89d4e8aa001f281ba80d6cf', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 172, 'total_tokens': 198}, 'model_name': 'Qwen/Qwen3-4B-Instruct-2507', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d3c499d9-d2dd-4fd7-9aa0-6e6fc9127f91-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 30}, 'id': 'chatcmpl-tool-ca922a3cc89d4e8aa001f281ba80d6cf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 172, 'output_tokens': 26, 'total_tokens': 198}),
 ToolMessage(content='6', name='multiply', tool_call_id='chatcmpl-tool-6710759eb1fb4425a4b3185f4ecf74bc')]

In [54]:
reply = llmWithTools.invoke(messages)

In [55]:
print(reply)
print(reply.content)

content='The product of 2 and 30 is 60.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 212, 'total_tokens': 227}, 'model_name': 'Qwen/Qwen3-4B-Instruct-2507', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='lc_run--3f6a14ac-dc1d-4ca8-94d5-120a3b636f6e-0' usage_metadata={'input_tokens': 212, 'output_tokens': 15, 'total_tokens': 227}
The product of 2 and 30 is 60.
